# Task 3
B06201053 數學四 鄭心慈

Price a maximum rainbow option with the payoff $\max\left(\max(S_{1T}, S_{2T},..., S_{nT})−K, 0\right)$ using the Monte Carlo simulatiton.

>Inputs: $K$, $r$, $T$, number of simulations, number of repetitions, $n$, $S_{10}$, $S_{20}$,..., $S_{n0}$, $q_1$, $q_2$,..., $q_n$, $\sigma_1$, $\sigma_2$,..., $\sigma_n$, $\rho_{ij}$, $1 \le i<j \le n$.

>Outputs: Option value and 95% confidence interval.

## Basic requirement (80 points)

Apply the Cholesky decomposition method to pricing the above rainbow option.

In [1]:
import numpy as np

In [2]:
### input

In [3]:
K=float(input('Please input K: '))

Please input K: 100


In [4]:
r=float(input('Please input r: '))

Please input r: 0.1


In [5]:
T=float(input('Please input T: '))

Please input T: 0.5


In [6]:
n_si=int(input('Please input number of simulations: '))

Please input number of simulations: 10000


In [7]:
n_re=int(input('Please input number of repetitions: '))

Please input number of repetitions: 20


In [8]:
n=int(input('Please input n: '))

Please input n: 5


In [9]:
S_0=list(map(float,input('Please input S_10,..., S_%d0: ' %n).strip().split()))[:n]

Please input S_10,..., S_50: 95 95 95 95 95


In [10]:
q=list(map(float,input('Please input q_1,..., q_%d: ' %n).strip().split()))[:n]

Please input q_1,..., q_5: 0.05 0.05 0.05 0.05 0.05


In [11]:
sigma=list(map(float,input('Please input sigma_1,..., sigma_%d: ' %n).strip().split()))[:n]

Please input sigma_1,..., sigma_5: 0.5 0.5 0.5 0.5 0.5


In [12]:
rho=np.identity(n)
for i in range(0, n-1, 1):
    rho[i,i+1:n]=list(map(float,input('Please input rho_%d%d,..., rho_%d%d: ' %(i+1,i+2,i+1,n)).strip().split()))[:n-i]

Please input rho_12,..., rho_15: 0.5 0.5 0.5 0.5
Please input rho_23,..., rho_25: 0.5 0.5 0.5
Please input rho_34,..., rho_35: 0.5 0.5
Please input rho_45,..., rho_45: 0.5


In [13]:
### Cholesky decomposition
def cholesky_decomposition(A):
    '''
    Arguments:
        A : 2D np.array

    Return:
        R : 2D np.array, A = R^T R
    '''
    assert len(A.shape) == 2
    m, n = A.shape
    assert m == n
    R = np.zeros([m, n])
    for i in range(0, m):
        for j in range(i, m):
            sum_d = sum(R[k, i] * R[k, j] for k in range(0, i))
            if i == j:
                R[i, j] = np.sqrt(A[i, j] - sum_d)
            else:
                R[i, j] = (A[i, j] - sum_d) / R[i, i]
    
    return R

In [14]:
### variance-covariance matrix
def C_matrix(T, n, sigma, rho):
    C=np.zeros([n,n])
    for i in range(0, n, 1):
        for j in range(0, n, 1):
            if i==j:
                C[i,j]=sigma[i]**2*T
            if i<j:
                C[i,j]=rho[i,j]*sigma[i]*sigma[j]*T
            else:
                C[i,j]=rho[j,i]*sigma[i]*sigma[j]*T
    return C

In [15]:
### standard normal sample
def Z_hat(n_si, n):
    Z_hat=np.zeros([n_si,n])
    for i in range(0, n, 1):
        Z_hat[:,i]=np.random.normal(0, 1, n_si)
    return Z_hat

In [16]:
def rainbow_option_price(K, r, T, n, S_0, q, sigma, rho, Z_hat):
    A=cholesky_decomposition(C_matrix(T, n, sigma, rho))
    r_hat=Z_hat@A
    mu=np.zeros(n)
    for i in range(0, n, 1):
        mu[i]=np.log(S_0[i])+(r-q[i]-sigma[i]**2/2)*T
    S_T_hat=np.exp(mu+r_hat)
    payoff=np.zeros(n_si)
    for i in range(0, n_si, 1):
        payoff[i]=max(max(S_T_hat[i])-K,0)
    price=np.exp(-r*T)*np.mean(payoff)
    return price

In [18]:
A=cholesky_decomposition(C_matrix(T, n, sigma, rho))
print(A)

[[0.35355339 0.1767767  0.1767767  0.1767767  0.1767767 ]
 [0.         0.30618622 0.10206207 0.10206207 0.10206207]
 [0.         0.         0.28867513 0.07216878 0.07216878]
 [0.         0.         0.         0.2795085  0.0559017 ]
 [0.         0.         0.         0.         0.27386128]]


In [17]:
### simulation
sample=np.zeros(n_re)
for i in range(0, n_re, 1):
    sample[i]=rainbow_option_price(K, r, T, n, S_0, q, sigma, rho, Z_hat(n_si, n))

print('The option value is', np.mean(sample))
print('The 95% confidence interval is','[', np.mean(sample)-2*np.std(sample), ',', np.mean(sample)+2*np.std(sample), ']')

The option value is 30.31891776050832
The 95% confidence interval is [ 29.903141340382962 , 30.734694180633678 ]


## Bonus 1 (5 points)

Combine the antithetic variate approach and moment matching method to price the above rainbow option.

In [ ]:
import math

In [ ]:
### standard normal sample(antithetic variate approach and moment matching method)
def Z_hat_1(n_si, n):
    Z_hat=np.zeros([n_si,n])
    for i in range(0, n, 1):
        Z_hat[0:math.ceil(n_si/2),i]=np.random.normal(0, 1, math.ceil(n_si/2))
        for j in range(math.ceil(n_si/2), n_si, 1):
            Z_hat[j,i]=-Z_hat[j-math.ceil(n_si/2),i]
    sigma_Z_hat=np.zeros(n)
    for i in range(0, n, 1):
         sigma_Z_hat[i]=np.std(Z_hat[:,i])
    Z_hat/=sigma_Z_hat
    return Z_hat

Z=np.empty((n_re, n_si, n))
for i in range(0,n_re,1):
    Z[i]=Z_hat_1(n_si, n)

In [ ]:
### simulation
sample=np.zeros(n_re)
for i in range(0, n_re, 1):
    sample[i]=rainbow_option_price(K, r, T, n, S_0, q, sigma, rho, Z[i])

print('The option value is', np.mean(sample))
print('The 95% confidence interval is','[', np.mean(sample)-2*np.std(sample), ',', np.mean(sample)+2*np.std(sample), ']')

## Bonus 2 (10 points)

Implement the inverse Cholesky method in Wang (2008) to price the above rainbow option.

In [ ]:
from numpy.linalg import inv

In [ ]:
def rainbow_option_price_1(K, r, T, n, S_0, q, sigma, rho, Z_hat):
    A=cholesky_decomposition(C_matrix(T, n, sigma, rho))
    B=cholesky_decomposition(np.cov(Z_hat,rowvar=False))
    r_hat=Z_hat@inv(B)@A
    mu=np.zeros(n)
    for i in range(0, n, 1):
        mu[i]=np.log(S_0[i])+(r-q[i]-sigma[i]**2/2)*T
    S_T_hat=np.exp(mu+r_hat)
    payoff=np.zeros(n_si)
    for i in range(0, n_si, 1):
        payoff[i]=max(max(S_T_hat[i])-K,0)
    price=np.exp(-r*T)*np.mean(payoff)
    return price

In [ ]:
### simulation
sample=np.zeros(n_re)
for i in range(0, n_re, 1):
    sample[i]=rainbow_option_price_1(K, r, T, n, S_0, q, sigma, rho, Z[i])

print('The option value is', np.mean(sample))
print('The 95% confidence interval is','[', np.mean(sample)-2*np.std(sample), ',', np.mean(sample)+2*np.std(sample), ']')

## Reference

Wang (2008), “Variance Reduction for Multivariate Monte Carlo Simulation,”  _Journal of Derivatives_ 16, pp. 7–28.